In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import json
import numpy as np
import cv2

from matplotlib import pyplot as plt
#import pandas as pd
from sklearn import svm
from sklearn.model_selection import GridSearchCV
#from skimage.io import imread

from skimage.io import imread
from skimage.transform import resize

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pickle

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

dir = "C:/food-101/food-101/images/"
categories = os.listdir(dir)

randomSampleCount = 1000

random.seed(255)
sequence = [i for i in range(1000)]
randList = random.sample(sequence, randomSampleCount)

randomSampled = []
scaleddir = 'C:/food-101/food-101/imagesScaled/'
#for cat in categories:
for cat in categories[:5]:
    directory = scaleddir + cat
    path = os.listdir(directory)
    index = 0
    imagesToBeSampled = []
    for images in path:
        index = index + 1
        if(index in randList):
            imagesToBeSampled.append(images)
    randomSampled.append(imagesToBeSampled)
#print(randomSampled[0])

### CNN ###
from sklearn.preprocessing import MinMaxScaler

scaled_X = []
images2 = []

scaleddir = 'C:/food-101/food-101/imagesScaled/'

index = 0
for cat in categories[:5]:
    directory = scaleddir + cat    
    path = randomSampled[index]
    index = index + 1
    for images in path:
        toArr = imread(os.path.join(directory, images))
        img_resized = resize(toArr, (100, 100,3))
        #flat.append(toArr)
        scaled_X.append(img_resized)
        images2.append(categories.index(cat))

scaled_X = np.array(scaled_X)


rs = 45
x_train, x_test, y_train, y_test = train_test_split(
    scaled_X, 
    images2, 
    test_size=0.3, 
    random_state=rs, 
    stratify=images2)

CNN Model Selection


In [3]:
# MODEL 1
# 4 LAYERS

model_cnn = models.Sequential(
    [
        layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (100,100,3)), #32 output channels, (3,3) convolution window
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'), #64 output channels, (3,3) convolution window
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'), #64 output channels, (3,3) convolution window
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'), #64 output channels, (3,3) convolution window
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'), #64 output channels, (3,3) convolution window
        
        layers.Flatten(),
        layers.Dense(64, activation = 'relu'),
        layers.Dense(5),

        
    ])
#model_cnn.summary()

## Convert data to tf tensors and train 

x_test_tensor = tf.convert_to_tensor(x_test)
x_train_tensor = tf.convert_to_tensor(x_train)

y_test_tensor = tf.convert_to_tensor(y_test)
y_train_tensor = tf.convert_to_tensor(y_train)

model_cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #SCC is optimal for multiclass
              metrics=['accuracy'])

history = model_cnn.fit(x_train_tensor, y_train_tensor, epochs=10, 
                    validation_data=(x_test_tensor, y_test_tensor))

#Validation Curves

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

print('Maximum Validation Accuracy = ', max(history.history['val_accuracy']))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          3

In [4]:
# MODEL 2 
# 2 LAYERS

model_cnn = models.Sequential(
    [
        layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (100,100,3)), #32 output channels, (3,3) convolution window
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'), #64 output channels, (3,3) convolution window
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'), #64 output channels, (3,3) convolution window
        
        layers.Flatten(),
        layers.Dense(64, activation = 'relu'),
        layers.Dense(5),

        
    ])

## Convert data to tf tensors and train

x_test_tensor = tf.convert_to_tensor(x_test)
x_train_tensor = tf.convert_to_tensor(x_train)

y_test_tensor = tf.convert_to_tensor(y_test)
y_train_tensor = tf.convert_to_tensor(y_train)

model_cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model_cnn.fit(x_train_tensor, y_train_tensor, epochs=10, 
                    validation_data=(x_test_tensor, y_test_tensor))

#Validation Curves

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

print('Maximum Validation Accuracy = ', max(history.history['val_accuracy']))

Epoch 1/10
125/125 [==============================] - 19s 153ms/step - loss: 1.3992 - accuracy: 0.3809 - val_loss: 1.3743 - val_accuracy: 0.4024
Epoch 2/10
125/125 [==============================] - 20s 159ms/step - loss: 1.1698 - accuracy: 0.5215 - val_loss: 1.1152 - val_accuracy: 0.5576
Epoch 3/10
125/125 [==============================] - 19s 154ms/step - loss: 1.0290 - accuracy: 0.5836 - val_loss: 1.1199 - val_accuracy: 0.5485
Epoch 4/10
125/125 [==============================] - 19s 154ms/step - loss: 0.8917 - accuracy: 0.6439 - val_loss: 1.1103 - val_accuracy: 0.5856
Epoch 5/10
125/125 [==============================] - 19s 153ms/step - loss: 0.7084 - accuracy: 0.7250 - val_loss: 1.1401 - val_accuracy: 0.5656
Epoch 6/10
125/125 [==============================] - 19s 151ms/step - loss: 0.5098 - accuracy: 0.8098 - val_loss: 1.3208 - val_accuracy: 0.5746
Epoch 7/10
125/125 [==============================] - 19s 149ms/step - loss: 0.3314 - accuracy: 0.8796 - val_loss: 1.5024 - val_ac

In [7]:
#MODEL 3
# 2 LAYERS, Increase output channels
model_cnn = models.Sequential(
    [
        layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (100,100,3)), #64 output channels, (3,3) convolution window
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128,(3,3), activation = 'relu'), #128 output channels, (3,3) convolution window
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128,(3,3), activation = 'relu'), #128 output channels, (3,3) convolution window
        
        layers.Flatten(),
        layers.Dense(64, activation = 'relu'),
        layers.Dense(5),

        
    ])

## Convert data to tf tensors and train

x_test_tensor = tf.convert_to_tensor(x_test)
x_train_tensor = tf.convert_to_tensor(x_train)

y_test_tensor = tf.convert_to_tensor(y_test)
y_train_tensor = tf.convert_to_tensor(y_train)

model_cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model_cnn.fit(x_train_tensor, y_train_tensor, epochs=10, 
                    validation_data=(x_test_tensor, y_test_tensor))

#Validation Curves

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

print('Maximum Validation Accuracy = ', max(history.history['val_accuracy']))

Epoch 1/10
125/125 [==============================] - 44s 352ms/step - loss: 1.5284 - accuracy: 0.3006 - val_loss: 1.3627 - val_accuracy: 0.4484
Epoch 2/10
125/125 [==============================] - 43s 341ms/step - loss: 1.2561 - accuracy: 0.4812 - val_loss: 1.1800 - val_accuracy: 0.5395
Epoch 3/10
125/125 [==============================] - 40s 317ms/step - loss: 1.1080 - accuracy: 0.5603 - val_loss: 1.2322 - val_accuracy: 0.4805
Epoch 4/10
125/125 [==============================] - 39s 315ms/step - loss: 1.0272 - accuracy: 0.5813 - val_loss: 1.1174 - val_accuracy: 0.5445
Epoch 5/10
125/125 [==============================] - 39s 315ms/step - loss: 0.9126 - accuracy: 0.6281 - val_loss: 1.1493 - val_accuracy: 0.5546
Epoch 6/10
125/125 [==============================] - 40s 318ms/step - loss: 0.7468 - accuracy: 0.7070 - val_loss: 1.1324 - val_accuracy: 0.5796
Epoch 7/10
125/125 [==============================] - 41s 327ms/step - loss: 0.6199 - accuracy: 0.7595 - val_loss: 1.3418 - val_ac